# LFM 梯度下降算法实现

### 0. 引入依赖


In [45]:
import numpy as np
import pandas as pd

### 1.数据准备

In [46]:
#评分矩阵R

R=np.array([[4,0,2,0,1],
    [0,2,3,0,0],
    [1,0,2,4,0],
    [5,0,0,3,1],
    [0,0,1,5,1],
    [0,3,2,4,1]])

R.shape


(6, 5)

### 2. 算法实现

In [47]:
"""
@输入参数：
R: M*N 的评分矩阵
K: 隐特征向量维度
max_iter: 最大迭代次数
alpha: 步长
lamda: 正则化系数

@输出
分解之后的P, Q
P: 初始化用户特征矩阵M*K
Q: 初始化物品特征矩阵N*K
"""

#给定超参数

K=5
max_iter = 5000
alpha = 0.0002
lamda=0.004

#核心算法
def LFM_grad_desc(R,max_iter,lamda, K=2,alpha=0.0001):
    #基本维度参数定义
    M=len(R)
    N=len(R[0])

    #P,Q初始值随机生成
    P=np.random.rand(M,K)
    Q=np.random.rand(N,K)
    Q=Q.T
    
    #开始迭代
    for steps in range(max_iter):
        #对所有的用户u物品i做遍历，对应的特征向量Pu，Qi梯度下降
        for u in range (M):
            for i in range(N):
                #对于每一个大于0的评分，求出预测评分误差
                if R[u][i]>0:
                    eui=np.dot(P[u],Q[:,i])-R[u][i]

                    #代入公式，按照梯度下降算法更新当前Pu和Qi
                    for k in range(K):
                        P[u][k]-=alpha*(2*eui*Q[k][i]+2*lamda*P[u][k])
                        Q[k][i]-=alpha*(2*eui*P[u][k]+2*lamda*Q[k][i])

    ##u,i遍历完成，所有特征向量更新完成，可以得到P，Q，可以计算预测评分矩阵
        predR=np.matmul(P,Q)

        ##计算当前损失函数
        cost=0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += (np.dot(P[u,:],Q[:,i])-R[u][i]) ** 2
                    #加上正则化项
                    for k in range(K):
                        cost+=lamda * (P[u][k] ** 2 + Q[k][i] ** 2 )
        
        if cost < 0.0001:
            break




    return P,Q.T, cost

### 3.测试

In [48]:
P,Q,cost = LFM_grad_desc(R,max_iter,lamda, K,alpha)

predR=P.dot(Q.T)

print(cost)
print(R)

predR

0.580495763939746
[[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]


array([[3.97762001, 2.22927031, 2.02007544, 4.68116326, 1.02992119],
       [4.08694258, 2.03841393, 2.95696321, 4.0383859 , 1.08428127],
       [1.014133  , 1.0766764 , 1.99481694, 3.98320788, 0.74713597],
       [4.99518596, 2.29731116, 2.57499481, 2.99482858, 1.0059497 ],
       [4.80407937, 2.16438049, 1.01952862, 4.99226814, 0.94197465],
       [4.15369296, 2.95913832, 1.99240253, 4.00579583, 1.02078038]])